In [0]:
import pandas as pd
import io
import glob

In [0]:
#wont work
check = pd.read_csv("/Volumes/workspace/default/sports/20240301-AllenField-1.csv")
#display(df)
csv_files = glob.glob("/Volumes/workspace/default/sports/*.csv")
print(csv_files)
#pwd

In [0]:
#df_spark = spark.read.csv("/Volumes/workspace/default/sports/*.csv", inferSchema=True, header=True)
#df_spark.columns
#df = df_spark.toPandas()

In [0]:
df = pd.read_csv("/Volumes/workspace/default/sports/20240301-AllenField-1.csv")

In [0]:
df[['Batter', 'Inning', 'Top/Bottom', 'Outs', 'Balls', 'Strikes', 'OutsOnPlay', 'PitchCall', 'PlayResult', 'RunsScored', 'KorBB']][:10]

In [0]:
#setting the initial values
df['b1'] = 0
df['b2'] = 0
df['b3'] = 0
df['b1Runner'] = None
df['b2Runner'] = None
df['b3Runner'] = None
df['Event'] = None
df['Error'] = '-'

In [0]:
base = 3
while base > 0:
    print(df.loc[10,'b'+str(base)+'Runner'], 'b'+str(base))
    base -= 1


In [0]:
def resetRunners(df, ind):
    # make changes to ind
    df.loc[ind, 'b1'] = 0
    df.loc[ind, 'b2'] = 0
    df.loc[ind, 'b3'] = 0
    df.loc[ind, 'b1Runner'] = None
    df.loc[ind, 'b2Runner'] = None
    df.loc[ind, 'b3Runner'] = None

#functions that update the next
def pushDown(df, ind):
    # make changes to ind+1
    # copy from previous row
    df.loc[ind+1, 'b1'] = df.loc[ind, 'b1']
    df.loc[ind+1, 'b2'] = df.loc[ind, 'b2']
    df.loc[ind+1, 'b3'] = df.loc[ind, 'b3']
    df.loc[ind+1, 'b1Runner'] = df.loc[ind, 'b1Runner']
    df.loc[ind+1, 'b2Runner'] = df.loc[ind, 'b2Runner']
    df.loc[ind+1, 'b3Runner'] = df.loc[ind, 'b3Runner']

def singleForced(df, ind):
    # make changes to ind+1
    # single forced for walk and HBP
    df.loc[ind+1, 'b1'] = 1
    df.loc[ind+1, 'b1Runner'] = df.loc[ind, 'Batter']

    if df.loc[ind, 'b1'] == 1:
        df.loc[ind+1, 'b2'] = 1
        df.loc[ind+1, 'b2Runner'] = df.loc[ind, 'b1Runner']
                
        if df.loc[ind, 'b2'] == 1:
            df.loc[ind+1, 'b3'] = 1
            df.loc[ind+1, 'b3Runner'] = df.loc[ind, 'b2Runner']

            if df.loc[ind, 'b3'] == 1:
                print(df.loc[ind, 'b3Runner'], " scored at ", ind)

def makeCorrections(df, ind, runs_scored):
    # make corrections to ind+1
    # how many base runner to remove depend on runs scored and outs on play
    # sacrifice batter removed from base 1
    # runs_scored - df.loc[ind, 'RunsScored'] + OutsOnPlay - scarifice
    # always done after updating the next row with changes
    count_base = df.loc[ind, 'b1'] + df.loc[ind, 'b2'] + df.loc[ind, 'b3']
    
    diff = df.loc[ind, 'RunsScored'] - runs_scored + df.loc[ind, 'OutsOnPlay']
    if df.loc[ind, 'PlayResult'] == 'Sacrifice':
        diff -= 1
        df.loc[ind+1, 'b1'] = 0
        df.loc[ind+1, 'b1Runner'] = None

    #basing on the assumption that base runner from base 3 are removed in order
    if diff > 0:
        print("***Corrections made for ", diff, " base runners at ", ind, " count ", count_base)
        df.loc[ind, 'Error'] = 'error'
        base = 3
        while diff > 0:
            if base < 1:
                print("*** error in calculation at ", ind)
            if df.loc[ind+1, 'b'+str(base)] == 1:
                df.loc[ind+1, 'b'+str(base)] = 0
                df.loc[ind+1, 'b'+str(base)+'Runner'] = None
                diff -= 1
            base -= 1
    

def single(df, ind):
    # make changes to ind+1
    # single playresult and sacrifise
    runs_scored = 0

    df.loc[ind+1, 'b1'] = 1
    df.loc[ind+1, 'b1Runner'] = df.loc[ind, 'Batter']

    if df.loc[ind, 'b1'] == 1:
        df.loc[ind+1, 'b2'] = 1
        df.loc[ind+1, 'b2Runner'] = df.loc[ind, 'b1Runner']
                
    if df.loc[ind, 'b2'] == 1:
        df.loc[ind+1, 'b3'] = 1
        df.loc[ind+1, 'b3Runner'] = df.loc[ind, 'b2Runner']

    if df.loc[ind, 'b3'] == 1:
        runs_scored += 1
        print(df.loc[ind, 'b3Runner'], " scored at ", ind)
    
    makeCorrections(df, ind, runs_scored)
    
def double(df, ind):
    # make changes to ind+1
    runs_scored = 0
    df.loc[ind+1, 'b2'] = 1
    df.loc[ind+1, 'b2Runner'] = df.loc[ind, 'Batter']

    if df.loc[ind, 'b1'] == 1:
        df.loc[ind+1, 'b3'] = 1
        df.loc[ind+1, 'b3Runner'] = df.loc[ind, 'b1Runner']

    if df.loc[ind, 'b2'] == 1:
        print(df.loc[ind, 'b2Runner'], " scored at ", ind)
        runs_scored += 1

    if df.loc[ind, 'b3'] == 1:
        print(df.loc[ind, 'b3Runner'], " scored at ", ind)
        runs_scored += 1
    
    makeCorrections(df, ind, runs_scored)

def homeRun(df, ind):
    # make changes to ind+1
    runs_scored = 1 + df.loc[ind, 'b1'] + df.loc[ind, 'b2'] + df.loc[ind, 'b3']
    print("run counted ", runs_scored, " check ", df.loc[ind, "RunsScored"])
    
    resetRunners(df, ind+1)
    


def triple(df, ind):
    # make changes to ind+1
    runs_scored = 0
    df.loc[ind+1, 'b3'] = 1
    df.loc[ind+1, 'b3Runner'] = df.loc[ind, 'Batter']

    if df.loc[ind, 'b1'] == 1:
        print(df.loc[ind, 'b1Runner'], " scored at ", ind)
        runs_scored += 1

    if df.loc[ind, 'b2'] == 1:
        print(df.loc[ind, 'b2Runner'], " scored at ", ind)
        runs_scored += 1

    if df.loc[ind, 'b3'] == 1:
        print(df.loc[ind, 'b3Runner'], " scored at ", ind)
        runs_scored += 1
    
    makeCorrections(df, ind, runs_scored)


In [0]:
#setting the initial values
df['b1'] = 0
df['b2'] = 0
df['b3'] = 0
df['b1Runner'] = None
df['b2Runner'] = None
df['b3Runner'] = None
df['Error'] = '-'

for ind in df.index[1:]:

    #recording the event
    if df.loc[ind, 'PitchCall'] == 'InPlay':
        df.loc[ind, 'Event'] = 'InPlay'
    if df.loc[ind, 'PlayResult'] == 'Out':
        df.loc[ind, 'Event'] += 'Out'
    if df.loc[ind, 'KorBB'] == 'Strikeout':
        df.loc[ind, 'Event'] = 'Strikeout'

    #inning change
    #updating the current row if change in inning
    if df.loc[ind, 'Top/Bottom'] != df.loc[ind-1, 'Top/Bottom'] \
    or df.loc[ind-1, 'Inning'] != df.loc[ind, 'Inning']:
        print('\tInning change at', ind)
        resetRunners(df, ind)

    #individual cases of runner moving update the next row
    #walk
    if df.loc[ind, 'KorBB'] == 'Walk':
        print("walk at ", ind)
        df.loc[ind, 'Event'] = 'Walk'
        singleForced(df, ind)
        #HBP
    elif df.loc[ind, 'PitchCall'] == 'HitByPitch':
        print("HBP at ", ind)
        df.loc[ind, 'Event'] = 'HBP'
        singleForced(df, ind)
    #single
    elif df.loc[ind, 'PlayResult'] == 'Single':
        print("single at ", ind)
        df.loc[ind, 'Event'] = 'Single'
        single(df, ind)
    elif df.loc[ind, 'PlayResult'] == 'Sacrifice':
        print("Sacrifice at ", ind)
        df.loc[ind, 'Event'] = 'Sacrifice'
        single(df, ind)

    #double
    elif df.loc[ind, 'PlayResult'] == 'Double':
        print("double at ", ind)
        df.loc[ind, 'Event'] = 'Double'
        double(df, ind)

    #triple
    elif df.loc[ind, 'PlayResult'] == 'Triple':
        print("triple at ", ind)
        df.loc[ind, 'Event'] = 'Triple'
        triple(df, ind)

    elif df.loc[ind, 'PlayResult'] == 'HomeRun':
        print("home run at ", ind)
        df.loc[ind, 'Event'] = 'HomeRun'
        homeRun(df, ind)

    #outs on play occured without any play being made
    #pushdown + makeing corrections so as to update runner for next row by makeCorrections    
    elif df.loc[ind, 'OutsOnPlay'] > 0 and df.loc[ind, 'PlayResult'] != 'Out':
        print("outs on play at ", ind)
        df.loc[ind, 'Event'] = 'OutsOnPlay'
        pushDown(df, ind)
        makeCorrections(df, ind, 0)
        
    #else
    #push down the base runners to next row
    else:
        pushDown(df, ind)

In [0]:
df[['Batter', 'b1', 'b2', 'b3', 'b1Runner', 'b2Runner', 'b3Runner', 'OutsOnPlay', 'Event', 'RunsScored', 'Error']][120:150]